In [ ]:
# # GDA test
# # f(x, y) = x^2 + y
# # nabla_x = 2x, nabla_y = 1
# test_GDA_update_step = GDA_update_step(alpha = 0.1, nabla_x= lambda x, y: 2 * x, nabla_y=lambda x, y: 1)

# get_timestep_images(x_0_lis = [10, 10, 7, 8], y_0_lis = [0, 1, 2, 3], update_function=test_GDA_update_step, steps=100, images_folder_path='images_test')

# gif_generation(images_num=100, 
#                images_folder_path='images_test')



In [ ]:
# # OGDA test
# # f(x, y) = x^2 + y
# # nabla_x = 2x, nabla_y = 1
test_GDA_update_step = OGDA_update_step(alpha = 0.1, nabla_x= lambda x, y: 2 * x, nabla_y=lambda x, y: 1)

get_timestep_images(x_0_lis = [10, 10, 7, 8], 
                    y_0_lis = [0, 1, 2, 3], 
                    update_function=test_GDA_update_step, 
                    steps=200, 
                    images_folder_path='images_test',
                    kernel="OGDA")

gif_generation(images_num=100, 
               images_folder_path='images_test')



In [ ]:
# figure 1- function
# f(x, y) = -1/8 x^2 - 1/2 y^2 + 6/10 xy
# nabla_x = -1/4 x + 6/10 y, nabla_y = -y + 6/10 x

nabla_x = lambda x, y: -1/4 * x + 6/10 * y
nabla_y = lambda x, y: -y + 6/10 * x
alpha = 0.01
x_0_lis = np.linspace(-0.25, 0.25, num=10)
y_0_lis = np.linspace(-0.25, 0.25, num=10) 
x_0_lis = list(x_0_lis) * 10
y_0_lis = list(y_0_lis) * 10
y_0_lis = sorted(y_0_lis)

steps = 1000
images_folder_path = 'figure_1'


In [6]:

test_GDA_update_step = GDA_update_step(alpha = alpha, 
                                       nabla_x= nabla_x,
                                        nabla_y=nabla_y)

test_OGDA_update_step = OGDA_update_step(alpha = alpha, 
                                       nabla_x= nabla_x,
                                        nabla_y=nabla_y)
get_timestep_images(x_0_lis = x_0_lis, 
                    y_0_lis = y_0_lis, 
                    update_function=test_OGDA_update_step, 
                    steps=steps,
                    images_folder_path=images_folder_path,
                    kernel="OGDA")

100%|██████████| 101/101 [00:24<00:00,  4.06it/s]


In [7]:
gif_generation(images_num=800,
                images_folder_path=images_folder_path,
                fps=steps//10, 
                step=10)


/tmp/ipykernel_66790/775396393.py:141: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f'{images_folder_path}/step_{i}.png'))
